# Loads and Imports

In [26]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [27]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

# Download and Import Latent Space data and corresponding Original Data from S3

In [28]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/treated0_l.p ../../temp/ldata_knn/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/match0_l.p ../../temp/ldata_knn/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/treated0_o.p ../../temp/odata_knn/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/match0_o.p ../../temp/odata_knn/match0_o.p

In [29]:
import pickle
with open(path+'/ldata_knn/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_knn/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata_knn/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_knn/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [30]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [31]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

$$d = |\frac{(\bar{x_t}-\bar{x_c})}{\sqrt{\frac{s_t^2 + s_c^2}{2}}}| $$

In [32]:
def ASMD_cont_calculate(tf, cf):
    t_mean = np.mean(tf.values)
    c_mean = np.mean(cf.values)
    t_var = np.var(tf.values)
    c_var = np.var(cf.values)
    d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
    return d

In [33]:
def ASMD_binary_calculate(tf, cf):
    pt = np.mean(tf.values)
    pc = np.mean(cf.values)
    d = np.abs((pt-pc)/(pt*(1-pt)+pc*(1-pc)))
    return d
    

In [34]:
def ASMD_final(cols, col_cat, cdf, tdf):
    ASMD = pd.DataFrame()
    cf = cdf[cols]
    tf = tdf[cols]
    for i, (col_cat_each, col_name_each) in enumerate(zip(col_cat, cols)):
        if col_cat_each == 'c':
            d = ASMD_cont_calculate(tf[col_name_each], cf[col_name_each])
        else:
            d = ASMD_binary_calculate(tf[col_name_each], cf[col_name_each])
        ASMD.loc[0, col_name_each] = d 
    return ASMD
    

### Latent Space

In [35]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [36]:
from itertools import repeat
given_value ='c'
latent_cols_cat=[]
latent_cols_cat.extend(repeat(given_value,len(latent_cols)))

In [37]:
ASMD = ASMD_final(latent_cols, latent_cols_cat, controls, treated)
ASMD

x1        x2        x3        x4       x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15       x16
0  0.014097  0.016401  0.020501  0.020111  0.03243  0.014621  0.017504  0.003815  0.015463  0.021979  0.020203  0.012191  0.016408  0.013938  0.014488  0.007054

### Original Space with George's Features only

In [40]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']
george_cols_cat = ['c', 'b', 'c', 'b', 'b', 'b', 'b']

In [41]:
ASMD = ASMD_final(george_cols, george_cols_cat, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.047872  0.368414   0.063845    0.024959    0.292091         0.099229        0.099633

### Original Space with all the features

In [47]:
original_cols = controls_orig.columns.to_list()[2:]

In [49]:
l1 = ['c', 'b', 'b', 'c', 'c', 'c']

given_value ='b'

orig_CC_cols_cat=[]
orig_CC_cols_cat.extend(repeat(given_value,len(original_cols)-len(l1)))

original_cols_cat = l1 + orig_CC_cols_cat

In [50]:
ASMD = ASMD_final(original_cols, original_cols_cat, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [52]:
ASMD.T

0
AGE_AT_MIDMONTH             0.047872
MBR_GNDR                    0.368414
MEDICAID_LOB                0.497279
ALLOW_AMT                   0.063845
ALLOW_ER                    0.015007
ALLOW_IP                    0.027246
CC_ADHD                     0.270238
CC_ALZHEIMER                1.001252
CC_ALZHEIMER_PLUS           0.783609
CC_AMI                      0.392434
CC_ANEMIA                   0.038736
CC_ANXIETY_DISORDER         0.072586
CC_ARTHRITIS                0.080084
CC_ASTHMA                   0.112902
CC_ATRIAL_FIB               0.048895
CC_AUTISM                   1.001449
CC_BIPOLAR                  0.030132
CC_BPH                      0.259077
CC_BREAST_CANCER            0.405341
CC_CATARACT                 0.068081
CC_CEREBRAL_PALSY           0.160345
CC_CHRONIC_KIDNEY           0.000148
CC_COLORECTAL_CANCER        0.210478
CC_COPD                     0.002070
CC_CYSTIC_FIBROSIS          0.561470
CC_DEPRESSION               0.038767
CC_DEPRESSIVE_DISORDERS     0.150062
CC_DEVELOP_DELAYS           0.270735
CC_DIABETES                 0.027203
CC_DISAB_DX_CNT             0.005116
CC_DX_CNT                   0.041306
CC_ENDOMETRIAL_CANCER       0.158371
CC_EPILEPSY                 0.139058
CC_FIBROMYALGIA             0.027148
CC_GLAUCOMA                 0.139072
CC_HEARING_IMPAIR           0.111655
CC_HEART_FAILURE            0.051137
CC_HEPATITIS_A              1.000258
CC_HEPATITIS_B_ACUTE        0.408459
CC_HEPATITIS_B_CHRONIC      0.341290
CC_HEPATITIS_C_ACUTE        1.000455
CC_HEPATITIS_C_CHRONIC      0.723820
CC_HEPATITIS_C_UNSPECIFIED  0.451090
CC_HEPATITIS_D                   NaN
CC_HEPATITIS_E                   NaN
CC_HEPATITIS_GEN            0.534861
CC_HIP_FRACTURE             1.000227
CC_HYPERLIPIDEMIA           0.038470
CC_HYPERTENSION             0.099229
CC_HYPOTHYROID              0.099633
CC_INT_DISAB                0.421384
CC_ISCHEMIC                 0.052584
CC_LEARN_DISAB              0.057852
CC_LEUKEMIAS                0.288058
CC_LIVER                    0.058392
CC_LUNG_CANCER              0.170101
CC_MIGRAINE                 0.201350
CC_MOBILITY_IMPAIR          0.128467
CC_MULTIPLE_SCLEROSIS       0.271786
CC_MUSCULAR_DYSTROPHY            NaN
CC_OBESITY                  0.292091
CC_OSTEOPOROSIS             0.022560
CC_PERIPHERAL               0.049924
CC_PERSONALITY              0.087996
CC_PRESSURE                 0.071926
CC_PROSTATE_CANCER          0.112149
CC_PTSD                     0.201084
CC_SCHIZOPHRENIA            0.261805
CC_SCHIZOPHRENIA_OTHER      0.381183
CC_SPINAL_CORD              0.359350
CC_SPINA_BIFIDA             0.309446
CC_STROKE                   0.097605
CC_TOBACCO                  0.024959
CC_TRAUMATIC_BRAIN          0.363179
CC_VISUAL_IMPAIR            0.093248